# Recommendation Using TF-IDF weighted Words Embedding

**STEP** <br>
1. Create TF-IDF 
2. Convert a tf-idf dictionary with word as key, idf as a value
3. Get TF-IDF features
4. Combine pretrained words embedding with TF-IDF
5. Calculate Cosine Similarity 
6. Recommend Law 

### Read ckiptagger & Dataframe

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import datetime
import pickle
from bs4 import BeautifulSoup
import jieba

path = "./ckip/data"
ws = WS(path)

df = pd.read_csv('../data/courses.csv').fillna('')
# Replace '@' with ' ' in original dataframe
#df.token = df.token.apply(lambda text: str(text).replace('@',' '))

2022-12-30 17:21:25.359932: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
2022-12-30 17:21:44.772817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, re

### Import Words Dictionary

### Read Pretrained Words Embedding
詞向量訓練文本來源為中文維基百科，全部的訓練文本可於[此](https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles.xml.bz2)下載最新版的中文維基百科。<br>
維基百科2014（總詞彙數：655K，400維詞向量，下載大小為2.5G）<br>
來源：[元智大學自然語言處理實驗室](http://nlp.innobic.yzu.edu.tw/demo/word-embedding.html)

In [2]:
token_list = np.load('./ckip/data/embedding_word/token_list.npy')
vector_list = np.load('./ckip/data/embedding_word/vector_list.npy')

embeddings = dict(zip(token_list, vector_list))
d = vector_list.shape[1]

In [3]:
course_df_original = pd.read_csv('../data/courses.csv').fillna('')
course_df = course_df_original.drop(['course_id', 'teacher_id', 'course_published_at_local', 'course_published_at_local', 'course_price'], axis=1)
course_df['description'] = course_df['description'].replace('([\<]).*?([\>])','',regex=True)
course_df = course_df.replace('\n', '',regex=True).replace('&.*;', '',regex=True).replace("--&?", "",regex=True)
course_df = course_df.replace('    ', ' ',regex=True).astype(str)
course_df = course_df.replace('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', ' ',regex=True).astype(str)
course_df.insert(0, 'course_id', course_df_original['course_id'])
course_df[:2]
info = []
for i in range(course_df.shape[0]):
    t = course_df.loc[i, :].values.flatten().tolist()
    info.append([t[0], ' '.join(t[:])])
info = pd.DataFrame(info, columns = ['course_id', 'text'])
info.iloc[33]['text']

'56385f4e423bdd0a00103c3c 怪獸工場 - 色彩光學篇 跟創作成為戀人~永遠的熱戀下去吧!10多年來的教學繪畫經驗分享給喜歡繪畫的朋友們！相關怪獸課程網址 @bluebobo3333?tr=bluebobo3333 藝術,設計 平面設計,色彩學,電腦繪圖 設計_平面設計,藝術_色彩學,藝術_電腦繪圖 photoshop,色彩光學,色彩學 講師簡介-藍國峰(直人Blue)怪獸FB- 、中階學員，所完整規劃的基礎課程。對於想手繪轉電繪、或是會基礎，卻無法快速掌握電繪技巧的你所規劃的課程。在灰階轉彩時，我們會遇到幾種最為常見的狀況，顏色過髒，配色過俗，沒有重點，跟場景的顏色搭不起來等...這些都是大家最為常見的困擾點。其實，方法很簡單，就差懂不懂當中的竅門而已！在此色彩光學篇，將會一步步的帶領大家，解決這些惱人的問題！了解光，探索顏色，完成一幅具有完整度的角色作品。首先我們必須先要了解光跟色彩的關係，當物體接觸到光後，會產生了哪些化學反應，當你了解了光，才能理解色彩運用的時機。當然，色彩心理學的基礎概念，是你一定需要必備的知識武器，首先，會先透過材質球及實例解析及演練，來了解色彩反應與功能操作應用方式。再來進入實際操作的階段，一步步的了解圖層與顏色之間的應用方式，藉由圖層強大的屬性，快速的達到心中理想的色彩變化。透過長年繪畫的經驗及累積，分享一些實用的操作技巧，讓角色或是在整體的情境氛圍掌控，都可以有著大大的加分效果！最後，會以真實的角色，做詳細的解說與拆解動作，讓同學們可以了解所學的技巧，如何予以應用在各種自己的創作中。戰谷松崗科技-馬雅之怒版權所有當我們可以靈活運用灰階及色彩篇所有的功能及觀念後，我們才可以談到所謂的『美感設計』。美感這環，其實是繪圖中最為困難的部分，必須要用主題單元的方式來做教學。大家若喜歡怪獸的教學，之後會以人物、角色、怪物、繪本、場景單元的方式，一一的深入分析及探討，讓大家可以更能找到專屬自己的美術價值。也懇請大家可以多多分享支持，幫怪獸集氣，一起與怪獸變得更強大！灰階上色寫實示範實例 會以完成一個可愛遊戲角色完稿為目標，若能完全了解課程中所提的觀念，是可以運用在各種商業所需的創作上。 Photoshop CS6+數位板 這是一套專門針對初、中階學員，完整規劃的基礎課程。著重於觀念的養成及功能的應用，只要是對電繪有興趣的學員

### Tf-idf for Tokenized Text in Dataframe

In [4]:
# TF-IDF Model
tfidf_ml = TfidfVectorizer(tokenizer=jieba.lcut)
tfidf_ml.fit(info.text)

# TF-IDF Dicitonary
dictionary = dict(zip(tfidf_ml.get_feature_names(), list(tfidf_ml.idf_)))

# feature name
tfidf_feature = tfidf_ml.get_feature_names()
'|'.join(tfidf_feature[5000:5050])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/6x/knkx1cnn26gf7h5w681lb4lw0000gs/T/jieba.cache
Loading model cost 0.697 seconds.
Prefix dict has been built successfully.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'lv2|lvm|lvmh|lydia|lydian|lymma|lynn|lyric|lyrics|m|m1|m5|m6|m88|ma|mac|macaron|macbook|macd|machine|macincloud|macleod|macos|macosx|macromicro|madame|madaster|made|madera|madker|magazine|maggie|maggiemay|magic|magicphotoshop2018|magnus|magritte|mai|mail|mailchimp|maine|mainly|maintenance|major|mak|make|makee|makeline|makepoint|maker'

### Newly Entered Text Preprocess function
- Remove Punctuation
- Remove Spaces
- Sentence Segment
- turn into list

In [5]:
def Preprocess(text):
    rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
    text = rule.sub(' ',str(text))
    #print(text)
    text = re.sub(' +', '',text)
    #text = ws([text],sentence_segmentation=True, recommend_dictionary=word_dict)
    text_lst = ws([text],sentence_segmentation=True)
    text_lst2 = [x for l in text_lst for x in l if len(x) > 1]
    #text_lst2.append(text)
    text_lst2 = list(set(text_lst2))
    redundant_phrase = ['更多', '觀念', '理念', '提升', '技能', '應用', '理論', '處理']
    for phrase in redundant_phrase:
        if(phrase in text_lst2):
            text_lst2.remove(phrase)
    #print(text_lst2)
    return text_lst2

### Calculate TF-IDF Weighted Word Embedding

In [6]:
starttime = datetime.datetime.now()

# TF-IDF weighted Word2Vec
tfidf_text_vect = [] # tfidf-w2v is stored in this list
row = 0

for text in info.text.apply(lambda text: text.split()):
    text_vect = np.zeros(300)
    weight_sum = 0
    for word in text:
        if word in embeddings.keys() and word in tfidf_feature:
            vec = embeddings[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tfidf_text_vect.append(text_vect)
    row += 1

# calculate running time
endtime = datetime.datetime.now()
print("建立模型時間: ",endtime - starttime)

建立模型時間:  0:00:33.470327


### Law Recommendation Function
輸入內文 --> 跑出推薦的前十個相近內文對應的法律

In [7]:
train = pd.read_csv('../data/train.csv')
courses = pd.read_csv('../data/courses.csv').fillna('')
bought_courses = {}
for id in train['course_id']:
    for i in id.split(' '):
        try:
            bought_courses[i] += 1
        except:
            bought_courses[i] = 1
    
bought_times = sorted(list(bought_courses.items()), key = lambda x:x[1], reverse=True)
garbage_course = list(set(courses['course_id']).difference(set(bought_courses.keys())))
popular_course_id = [x[0] for x in bought_times] + garbage_course
top_10 = popular_course_id[:10]
top_10

['5fc5ee1b08b74a6e3723abd2',
 '5f7c210b1de7982fb413a3e9',
 '6030c9cd99e14cc2401e66b9',
 '5f7c209762ad22756c7a1c74',
 '60cb0a440dabda80019d5f7c',
 '5f7c212262ad2203e77a1cc9',
 '5ef099ab678184065fd4d426',
 '6059aee039f2512548c187c6',
 '5ef0a8a5775dea0566e6cb5f',
 '5ef0a2198e8212c4e6335387']

In [8]:
def recommend_law(text, tfidf_text_vect = tfidf_text_vect):
    top_k = 50
    if(text.replace(' ', '') == ''):
        return popular_course_id[:top_k]
    text = Preprocess(text)
    text_vect = np.zeros(300) # w2v size
    weight_sum = 0
    for word in text:
        if word in embeddings.keys() and word in tfidf_feature:
            vec = embeddings[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tmp_vect = [*tfidf_text_vect,text_vect]
    new_cos_sim = cosine_similarity(tmp_vect, tmp_vect)
    #print(len(tfidf_text_vect))
    sim_score = np.sort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:top_k+1]
    try:
        tmp_top_10_law = info[['text']].iloc[np.argsort(new_cos_sim[(new_cos_sim.shape[0]-1)])[::-1][1:top_k+1]]
    except IndexError:
        tmp_top_10_law = info[['text']].iloc[np.argsort(new_cos_sim[(new_cos_sim.shape[0]-1)])[::-1][2:top_k+2]]

        
    tmp_top_10_law['similarity_score'] = [round(score*100,1) for score in sim_score]
    return tmp_top_10_law

### Try an Example
輸入內容便可以推薦出適合的法律<br>

In [18]:
starttime = datetime.datetime.now()

newtext = '''
設計_平面設計 程式_程式語言 設計_動態設計 程式_網頁前端
'''



result = recommend_law(newtext)

# calculate running time
endtime = datetime.datetime.now()
print("搜尋推薦時間: ",endtime - starttime)
pd.DataFrame(result['text'].apply(lambda x:x[24:]))

# 資訊科技,科技業 
# 語言_英文,攝影_商業攝影,行銷_數位行銷,程式_程式入門,程式_程式語言,設計_網頁設計,職場技能_職場溝通,職場技能_資料彙整,藝術_電腦繪圖,藝術_繪畫與插畫,職場技能_更多職場技能,生活品味_數學,語言_日文,生活品味_運動,藝術_手寫字,攝影_影像創作,攝影_後製剪輯,設計_應用設計,職場技能_效率提升,藝術_角色設計,程式_量化分析,生活品味_心靈成長與教育
# 區塊鏈,手寫字,手遊,插畫,旅行旅遊,水彩,素描,速寫,運動健身,閱讀,電影,電玩,電腦繪圖

# 60ddc3ca06259d00064c7f17 60aeac37bca91777bf5bb114 60c84de9eb75ca46e0c25e85 611f7d91bd122100071f2926 611f5d074b76af0007c24d7e 6155cda6d425f500065f5c96 6156a77fdf426a0007cc5fe1


搜尋推薦時間:  0:00:00.068935


,text
70,動畫互動網頁程式入門 (HTML/CSS/JS) 墨雨設計創辦人，紐約大學進修整合數位媒體...
268,"為個人品牌設計的 WiX 網站架設實作 Hi, 我們是Brenda和Winnie，以前我們..."
659,N 、MongoDB 網站後端工程入門 喜歡技術，更喜歡與人互動，不小心，在講台上教學超過...
608,職場英文大補帖｜銀行行員必備的英文口說 CLN 致力於引領企業用外語佈局全球、教導學員以外...
336,AutoLisp - 紮實入門與實例應用 程式應用專業、知識轉述興趣。電腦程式及繪圖多年經...
615,入門數據分析，掌握 Hive SQL 取數能力 數據領域資深從業者，<Rock Data>...
511,數位轉型敲門磚，產業必學 AI 思維通識課 陳縕儂老師的研究專長包含對話系統、語言理解、深...
224,從生活認識微積分：基礎觀念篇(1) 大學時專攻生命科學與數學。對於科學教育很有熱誠，目前已...
338,開啟程式設計之門 - C++ 核心課程 22 歲的 Kevin 目前就讀於交通大學。念的是...
665,建立個人品牌視覺：諾米帶你上手風格再造 Today at Apple 是由 Apple 精...


In [10]:
users = pd.read_csv('../data/users.csv').fillna('')
user_popular_interest = {}
for user, interests in zip(users['user_id'], users['interests']):
    for interest in interests.split(','):
        if(interest != ''):
            try:
                user_popular_interest[interest] += 1
            except:
                user_popular_interest[interest] = 1
user_popular_interest_list = sorted(list(user_popular_interest.items()), key=lambda x:x[1], reverse=True)
user_popular_interest_list = [x[0] for x in user_popular_interest_list]
user_popular_interest_list.append('')
user_popular_interest_list

['語言_英文',
 '投資理財_理財',
 '設計_平面設計',
 '藝術_繪畫與插畫',
 '投資理財_投資觀念',
 '行銷_數位行銷',
 '藝術_電腦繪圖',
 '攝影_影像創作',
 '攝影_後製剪輯',
 '語言_日文',
 '職場技能_職場溝通',
 '生活品味_烹飪料理與甜點',
 '行銷_文案',
 '職場技能_資料彙整',
 '職場技能_效率提升',
 '行銷_社群行銷',
 '設計_網頁設計',
 '投資理財_金融商品',
 '攝影_商業攝影',
 '職場技能_文書處理',
 '職場技能_個人品牌經營',
 '生活品味_心靈成長與教育',
 '程式_程式入門',
 '生活品味_運動',
 '程式_程式語言',
 '職場技能_創業',
 '手作_手作小物',
 '設計_動態設計',
 '藝術_手寫字',
 '生活品味_壓力舒緩',
 '藝術_角色設計',
 '程式_網頁前端',
 '設計_應用設計',
 '語言_韓文',
 '藝術_色彩學',
 '音樂_樂器',
 '手作_更多手作',
 '攝影_動態攝影',
 '設計_介面設計',
 '職場技能_產品設計',
 '人文_文學',
 '藝術_字體設計',
 '藝術_更多藝術',
 '程式_量化分析',
 '設計_設計理論',
 '生活品味_護膚保養與化妝',
 '人文_社會科學',
 '生活品味_更多生活品味',
 '程式_網站架設',
 '程式_資料科學',
 '職場技能_求職',
 '攝影_影視創作',
 '職場技能_更多職場技能',
 '程式_程式理財',
 '音樂_音樂創作',
 '藝術_素描',
 '生活品味_寵物',
 '行銷_更多行銷',
 '投資理財_更多投資理財',
 '程式_手機程式開發',
 '音樂_人聲',
 '手作_刺繡',
 '程式_網頁後端',
 '程式_程式思維',
 '攝影_攝影理論',
 '生活品味_親子教育',
 '程式_區塊鏈',
 '投資理財_量化交易',
 '語言_西班牙文',
 '攝影_更多攝影',
 '設計_更多設計',
 '語言_歐洲語言',
 '手作_手工書',
 '程式_資訊安全',
 '手作_手工印刷',
 '音樂_音樂理論',
 '程式_遊戲開發',
 '程式_軟體程式開發與維護',
 '人文_更多人文',
 '投資理財_比特幣',
 '

In [11]:
user_dict = {}
for user, title, interest, name in zip(users['user_id'], users['occupation_titles'],users['interests'],users['recreation_names']):
    sub_only = []
    #user_dict[user] = ' '.join(title.split(',')) + ' ' + ' '.join(interest.split(',')) + ' ' + ' '.join(name.split(','))
    interest_lst = interest.split(',')
    user_dict[user] = ' '.join(sorted(interest_lst, key=user_popular_interest_list.index)[:min(5,len(interest_lst))])\
                                    + ' ' + ' '.join(name.split(','))

list(user_dict.items())[50:100]

[('54f1ecf24ec3c809002e4a6c',
  '藝術_繪畫與插畫 藝術_電腦繪圖 攝影_後製剪輯 生活品味_心靈成長與教育 手作_手作小物 '),
 ('54f2c1ed4ec3c809002e4a84', '設計_平面設計 藝術_繪畫與插畫 藝術_電腦繪圖 藝術_角色設計 '),
 ('54f2c8644ec3c809002e4a92', ' '),
 ('54f2f6ff4ec3c809002e4ab0',
  '設計_平面設計 藝術_繪畫與插畫 藝術_電腦繪圖 藝術_角色設計 手作 手寫字 插畫 水彩 貓派 速寫 閱讀 電影 電玩 電腦繪圖'),
 ('54f305134ec3c809002e4ab7',
  '設計_平面設計 藝術_繪畫與插畫 藝術_電腦繪圖 生活品味_心靈成長與教育 藝術_色彩學 插畫 插花 水彩 狗派 瑜珈 素描 貓派 電腦繪圖'),
 ('54f327814ec3c809002e4ace', '投資理財_理財 藝術_繪畫與插畫 投資理財_投資觀念 藝術_電腦繪圖 藝術_色彩學 '),
 ('54f3c2104ec3c809002e4ae7', '語言_韓文 藝術_更多藝術 '),
 ('54f3cd824ec3c809002e4aec', '藝術_繪畫與插畫 藝術_角色設計 人文_文學 手作 插畫 電腦繪圖'),
 ('54f3d2ee4ec3c809002e4af2', '設計_平面設計 藝術_繪畫與插畫 藝術_電腦繪圖 職場技能_職場溝通 職場技能_文書處理 '),
 ('54f3ffe54ec3c809002e4b06',
  '設計_平面設計 藝術_繪畫與插畫 行銷_數位行銷 藝術_電腦繪圖 攝影_影像創作 占卜 插畫 速寫 閱讀'),
 ('54f402584ec3c809002e4b09',
  '藝術_繪畫與插畫 行銷_數位行銷 攝影_影像創作 攝影_後製剪輯 職場技能_職場溝通 寫作 手遊 插畫 政治經濟 旅行旅遊 運動賽事 閱讀 電影 電玩 電腦繪圖 電視劇'),
 ('54f48a954ec3c809002e4b3e',
  '行銷_數位行銷 職場技能_職場溝通 職場技能_資料彙整 設計_網頁設計 職場技能_文書處理 '),
 ('54f498364ec3c809002e4b4b',
  '語言_英

In [12]:
# test_unseen = pd.read_csv('../data/test_unseen.csv')
# with open('./submit_unseen_course.csv', 'w') as f:
#     f.write('user_id,course_id\n')
#     st_time = datetime.datetime.now()
#     for i, user in enumerate(test_unseen['user_id']):
#         t = top_10.copy()
#         if(i % 500 == 0):
#             ed_time = datetime.datetime.now()
#             print(i, "花費時間: ", ed_time - st_time)
#         for course in recommend_law(user_dict[user])['text']: 
#             t.append(course[:24])
#             if(len(t) == 50):
#                 break
#         #print(len(t))
#         f.write('{},{}\n'.format(user, ' '.join(t)))
# print('finish')